# 1. Set Up Environment

In [1]:
#Import Libraries
import fitz #library to load pdf files
import pandas as pd #library to transform data
import re #library to leverage regex

In [2]:
# Set pandas display options
pd.set_option('display.max_rows', None)

# Turn off deprecation warnings from pandas
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 2. Load Data

In [3]:
#Load 2023 assessment
albany_assessment_2023 = 'original_data/Albany 2023 Final Assessme.pdf' #29,595 records

#Load 2024 assessment
albany_assessment_2024 = 'original_data/Albany 2024 Final Roll.pdf' #29,589 records

#Load test dataset
test_data = 'original_data/Albany 2023 Final Assessment_partial.pdf' #275 records

#Pseudo-code
1. Iterate through pages
2. Separate Records
3. 

# 3. By page scans (TEST)

In [4]:
def extract_property_tax_data_by_page(pdf_document):
    #regex
    delimiter_pattern = r'(?:\*+ )(?:\d{1,2}\.\d{1,2}-\d*?-)(?:(?:\d{1,2}\.\d{1,2})|(?:\d{1,2}))(?: \*+)' #regex to find start of record (orde delimit records)
    parcelnum_pattern = r'(?:\d{1,2}\.\d{1,2}-\d*?-)(?:(?:\d{1,2}\.\d{1,2})|(?:\d{1,2}))' #regex to find parcel number
    page_header_pattern = r'STATE OF NEW YORK[\s\S]*?ACCOUNT NO\.\s+' #to be removed from each page
    address_pattern = r'.+\s+(?=(?:HOMESTEAD PARCEL|NON-HOMESTEAD PARCEL))'
    property_type_pattern = r'(?:(?:\d{1,2}\.\d{1,2}-\d*?-)(?:(?:\d{1,2}\.\d{1,2})|(?:\d{1,2})))(?: \n \n +)(\d{3} )' #r'(?: \n \n +)(\d{3} )' #r'(?: +)(\d{3} )'
    county_tax_pattern = r'(?:COUNTY  TAXABLE VALUE\s+)(\b(\d{1,4}(,\d{3})+)\b|[1-9]\d+|0)'
    city_tax_pattern = r'(?:CITY    TAXABLE VALUE\s+)(\b(\d{1,4}(,\d{3})+)\b|[1-9]\d+|0)'
    full_market_value_pattern = r'(?:FULL MARKET VALUE\s+)(\b(\d{1,4}(,\d{3})+)\b|[1-9]\d+)'
    school_tax_pattern = r'(?:SCHOOL  TAXABLE VALUE\s+)(\b(\d{1,4}(,\d{3})+)\b|[1-9]\d+|0)'
    
    
    #load pdf doc
    document = fitz.open(pdf_document)

    # Initialize an empty list to store matching text
    parcel_numbers_lst = []
    address_lst = []
    re_type_lst = [] #eg 210, 220, 311, etc
    county_tax_value_list = []
    city_tax_value_list = []
    full_market_value_list = []
    school_tax_value_list = []
    lists_to_check = [parcel_numbers_lst, address_lst, re_type_lst, county_tax_value_list, city_tax_value_list, full_market_value_list, school_tax_value_list]



    #iterate through each pdf page
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text = page.get_text("text")

        #remove page headers using regex
        text = re.sub(page_header_pattern, '', text)

        # #view text
        # if counter < 2:
        #     print(text)
        # counter += 1 
            

        #get parcel numbers
        matches_parcel = re.findall(parcelnum_pattern, text) #find all parcel numbers
        #print(matches_parcel)
        matches_parcel = matches_parcel[::2]
        parcel_numbers_lst.extend(matches_parcel) #append parcel numbers to list

        #get address
        matches_address = re.findall(address_pattern, text)
        #print(matches_address)
        address_lst.extend(matches_address) #append address to list

        #get property type code
        matches_prop_type = re.findall(property_type_pattern, text)
        #print(matches_prop_type)
        re_type_lst.extend(matches_prop_type) #property_type_code address to list

        #get county taxable value
        matches_county_tax_value = re.findall(county_tax_pattern, text)
        #print(matches_county_tax_value[0][0])
        for item in matches_county_tax_value:
            county_tax_value_list.append(''.join(item[0]))

        #get city taxable value
        matches_city_tax_value = re.findall(city_tax_pattern, text)
        #print(matches_city_tax_value[0][0])
        for item in matches_city_tax_value:
            city_tax_value_list.append(''.join(item[0]))

        #get full market value
        matches_full_market_value = re.findall(full_market_value_pattern, text)
        #print(matches_city_tax_value[0][0])
        for item in matches_full_market_value:
            full_market_value_list.append(''.join(item[0]))

        #get school tax value
        matches_school_tax_value = re.findall(school_tax_pattern, text)
        for item in matches_school_tax_value:
            #print(item)
            school_tax_value_list.append(''.join(item[0]))
    

    #view regex match:
    # print(len(county_tax_value_list))
    # for item in county_tax_value_list:
    #     print(item)
    print(f'There are {len(address_lst)} parcels')
    if len(parcel_numbers_lst) == len(address_lst):
        print(f'The length of parcels is {len(parcel_numbers_lst)} whereas the length of addresses is {len(address_lst)}. You extracted all addresses correctly')
    else:
        print('They are different lengths. Address Extraction Failed.')
        
    if len(parcel_numbers_lst) == len(re_type_lst):
        print(f'The length of parcels is {len(parcel_numbers_lst)} whereas the length of Property Type Code is {len(re_type_lst)}. You extracted all Property Type Codes correctly')
    else:
        print(f'They are different lengths. Property Type Code Extraction Failed. There are {len(re_type_lst)} counts.')

    if len(parcel_numbers_lst) == len(county_tax_value_list):
        print(f'The length of parcels is {len(parcel_numbers_lst)} whereas the length of County Tax Value is {len(county_tax_value_list)}. You extracted all County Tax Values correctly')
    else:
        print(f'They are different lengths. County Tax Value Extraction Failed. There are {len(county_tax_value_list)} counts.')

    if len(parcel_numbers_lst) == len(city_tax_value_list):
        print(f'The length of parcels is {len(parcel_numbers_lst)} whereas the length of City Tax Value is {len(city_tax_value_list)}. You extracted all City Tax Values correctly')
    else:
        print(f'They are different lengths. City Tax Value Extraction Failed. There are {len(city_tax_value_list)} counts.')

    if len(parcel_numbers_lst) == len(full_market_value_list):
        print(f'The length of parcels is {len(parcel_numbers_lst)} whereas the length of Full Market Value is {len(full_market_value_list)}. You extracted all Full Market Values correctly')
    else:
        print(f'They are different lengths. Full Market Value Extraction Failed. There are {len(full_market_value_list)} counts.')

    if len(parcel_numbers_lst) == len(school_tax_value_list):
        print(f'The length of parcels is {len(parcel_numbers_lst)} whereas the length of School Tax Value is {len(school_tax_value_list)}. You extracted all School Tax Values correctly')
    else:
        print(f'They are different lengths. School Tax Value Extraction Failed. There are {len(school_tax_value_list)} counts.')


    #create table with extracted values
    if all(len(lst) == len(parcel_numbers_lst) for lst in lists_to_check):
        #creat a dict with the lists
        #print(len(item))
        data = {
            'parcel_num': parcel_numbers_lst,
            'address': address_lst,
            'property_type_code': re_type_lst,
            'county_tax': county_tax_value_list,
            'city_tax': city_tax_value_list,
            'full_market_value': full_market_value_list,
            'school_tax': school_tax_value_list
        }

        #create dataframe
        df = pd.DataFrame(data)
        return df
    else:
        return "Not all lists are of the same length as the original_list."

        
df = extract_property_tax_data_by_page(test_data)

df.head()

There are 275 parcels
The length of parcels is 275 whereas the length of addresses is 275. You extracted all addresses correctly
The length of parcels is 275 whereas the length of Property Type Code is 275. You extracted all Property Type Codes correctly
The length of parcels is 275 whereas the length of County Tax Value is 275. You extracted all County Tax Values correctly
The length of parcels is 275 whereas the length of City Tax Value is 275. You extracted all City Tax Values correctly
The length of parcels is 275 whereas the length of Full Market Value is 275. You extracted all Full Market Values correctly
The length of parcels is 275 whereas the length of School Tax Value is 275. You extracted all School Tax Values correctly


,parcel_num,address,property_type_code,county_tax,city_tax,full_market_value,school_tax
0,76.22-3-40,Rear 335.5 Myrtle Ave,311,"1,400","1,400","1,977","1,400"
1,75.36-2-79,3 Academy Rd \n \n,210,"205,000","205,000","289,425","205,000"
2,75.36-2-78,5 Academy Rd \n \n,210,"162,000","90,000","254,130","97,110"
3,75.36-2-77,7 Academy Rd \n \n,210,"237,000","237,000","334,604","237,000"
4,75.36-2-76,9 Academy Rd \n \n,210,"192,000","192,000","271,072","168,080"


# 4. By Record

In [5]:
def extract_property_tax_data_by_record(pdf_document):
    #regex
    page_header_pattern = r'STATE OF NEW YORK[\s\S]*?ACCOUNT NO\.\s+' #to be removed from each page
    delimiter_pattern =  r'(\*+ \d{1,2}.\d{1,2}(?:-\d+-)(?:(?:\d{1,2}.\d{1,2})|\d+) \*+)' 
    parcelnum_pattern = r'(?:\d{1,2}\.\d{1,2}-\d*?-)(?:(?:\d{1,2}\.\d{1,2})|(?:\d{1,2}))' #regex to find parcel number
    address_pattern = r'.+\s+(?=(?:HOMESTEAD PARCEL|NON-HOMESTEAD PARCEL))'
    property_type_pattern = r'(?:(?:\d{1,2}\.\d{1,2}-\d*?-)(?:(?:\d{1,2}\.\d{1,2})|(?:\d{1,2})))(?: \n \n +)(\d{3} )' #r'(?: \n \n +)(\d{3} )' #r'(?: +)(\d{3} )'
    county_tax_pattern = r'(?:COUNTY  TAXABLE VALUE\s+)(\b(\d{1,4}(,\d{3})+)\b|[1-9]\d+|0)'
    city_tax_pattern = r'(?:CITY    TAXABLE VALUE\s+)(\b(\d{1,4}(,\d{3})+)\b|[1-9]\d+|0)'
    full_market_value_pattern = r'(?:FULL MARKET VALUE\s+)(\b(\d{1,4}(,\d{3})+)\b|[1-9]\d+)'
    school_tax_pattern = r'(?:SCHOOL  TAXABLE VALUE\s+)(\b(\d{1,4}(,\d{3})+)\b|[1-9]\d+|0)'

    #load pdf doc
    document = fitz.open(pdf_document)

    #initialize string to hold all text
    all_text = ""

    #iterate through each pdf page and append it to a text string
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text = page.get_text("text")

        #remove page headers using regex
        text = re.sub(page_header_pattern, '', text)
        all_text += text

    #slice text into records
    split_text = re.split(delimiter_pattern, all_text) #Delimit text by record
    records = [record.strip() for record in split_text if record.strip()] #remove empty strings from the list
    combined_records = [records[i] + "\n" + records[i+1] for i in range(0, len(records), 2)] #combine pairs of items in the list

    #get data from each record:
    # Initialize an empty list to store matching text
    parcel_numbers_lst = []
    address_lst = []
    re_type_lst = [] #eg 210, 220, 311, etc
    county_tax_value_list = []
    city_tax_value_list = []
    full_market_value_list = []
    school_tax_value_list = []
    lists_to_check = [parcel_numbers_lst, address_lst, re_type_lst, county_tax_value_list, city_tax_value_list, full_market_value_list, school_tax_value_list]

    #create an empty dataframe
    df = pd.DataFrame({
        'parcel_num': parcel_numbers_lst,
        'address': address_lst,
        'property_type_code': re_type_lst,
        'county_tax': county_tax_value_list,
        'city_tax': city_tax_value_list,
        'full_market_value': full_market_value_list,
        'school_tax': school_tax_value_list
        })
    
    for text in combined_records:
        #get parcel numbers
        matches_parcel = re.findall(parcelnum_pattern, text) #find all parcel numbers
        matches_parcel = matches_parcel[::2]
        if not matches_parcel:
            matches_parcel = [""]

        #get address
        matches_address = re.findall(address_pattern, text)
        if not matches_address:
            matches_address = [""]

        #get property type code
        matches_prop_type = re.findall(property_type_pattern, text)
        if not matches_prop_type:
            matches_prop_type = [""]

        #get county taxable value
        matches_county_tax_value = re.findall(county_tax_pattern, text)
        county_tax_values = [''.join(item[0]) for item in matches_county_tax_value] if matches_county_tax_value else [""]

        #get city taxable value
        matches_city_tax_value = re.findall(city_tax_pattern, text)
        city_tax_values = [''.join(item[0]) for item in matches_city_tax_value] if matches_city_tax_value else [""]

        #get full market value
        matches_full_market_value = re.findall(full_market_value_pattern, text)
        #print(matches_city_tax_value[0][0])
        full_market_values = [''.join(item[0]) for item in matches_full_market_value] if matches_full_market_value else [""]

        #get school tax value
        matches_school_tax_value = re.findall(school_tax_pattern, text)
        school_tax_values = [''.join(item[0]) for item in matches_school_tax_value] if matches_school_tax_value else [""]

        # Combine the lists into rows and append to the DataFrame
        for i in range(len(matches_parcel)):
            row = {
                'parcel_num': matches_parcel[i] if i < len(matches_parcel) else "",
                'address': matches_address[i] if i < len(matches_address) else "",
                'property_type_code': matches_prop_type[i] if i < len(matches_prop_type) else "",
                'county_tax': county_tax_values[i] if i < len(county_tax_values) else "",
                'city_tax': city_tax_values[i] if i < len(city_tax_values) else "",
                'full_market_value': full_market_values[i] if i < len(full_market_values) else "",
                'school_tax': school_tax_values[i] if i < len(school_tax_values) else ""
                }

        df = df.append(row, ignore_index=True)
    
    return df


#print(len(extract_property_tax_data_by_record(test_data)))



## 2023 Data

In [6]:
df_albany_2023 = extract_property_tax_data_by_record(albany_assessment_2023)
df_albany_2023.head()

,parcel_num,address,property_type_code,county_tax,city_tax,full_market_value,school_tax
0,76.22-3-40,Rear 335.5 Myrtle Ave,311,"1,400","1,400","1,977","1,400"
1,75.36-2-79,3 Academy Rd \n \n,210,"205,000","205,000","289,425","205,000"
2,75.36-2-78,5 Academy Rd \n \n,210,"162,000","90,000","254,130","97,110"
3,75.36-2-77,7 Academy Rd \n \n,210,"237,000","237,000","334,604","237,000"
4,75.36-2-76,9 Academy Rd \n \n,210,"192,000","192,000","271,072","168,080"


In [7]:
#Save data to parquet file locally
df_albany_2023.to_parquet('parsed_data/albany_assessment_2023.parquet', index=False)

## 2024 Data

In [8]:
df_albany_2024 = extract_property_tax_data_by_record(albany_assessment_2024)
df_albany_2024.head()

,parcel_num,address,property_type_code,county_tax,city_tax,full_market_value,school_tax
0,76.22-3-40,Rear 335.5 Myrtle Ave,311,"3,000","3,000","3,000","3,000"
1,75.36-2-79,3 Academy Rd \n \n,210,"378,000","378,000","378,000","378,000"
2,75.36-2-78,5 Academy Rd \n \n,210,"149,000","149,000","298,000","65,000"
3,75.36-2-77,7 Academy Rd \n \n,210,"334,000","334,000","334,000","334,000"
4,75.36-2-76,9 Academy Rd \n \n,210,"359,000","359,000","359,000","329,000"


In [9]:
#Save data to parquet file locally
df_albany_2024.to_parquet('parsed_data/albany_assessment_2024.parquet', index=False)

## Combine 2023 and 2024 data

In [10]:
# Rename columns to distinguish between 2023 and 2024
df_albany_2023 = df_albany_2023.rename(columns=lambda x: x + '_2023' if x != 'parcel_num' else x)
df_albany_2024 = df_albany_2024.rename(columns=lambda x: x + '_2024' if x != 'parcel_num' else x)

#merge data
merged_data = pd.merge(df_albany_2023, df_albany_2024, on='parcel_num')
merged_data.head()

,parcel_num,address_2023,property_type_code_2023,county_tax_2023,city_tax_2023,full_market_value_2023,school_tax_2023,address_2024,property_type_code_2024,county_tax_2024,city_tax_2024,full_market_value_2024,school_tax_2024
0,76.22-3-40,Rear 335.5 Myrtle Ave,311,"1,400","1,400","1,977","1,400",Rear 335.5 Myrtle Ave,311,"3,000","3,000","3,000","3,000"
1,75.36-2-79,3 Academy Rd \n \n,210,"205,000","205,000","289,425","205,000",3 Academy Rd \n \n,210,"378,000","378,000","378,000","378,000"
2,75.36-2-78,5 Academy Rd \n \n,210,"162,000","90,000","254,130","97,110",5 Academy Rd \n \n,210,"149,000","149,000","298,000","65,000"
3,75.36-2-77,7 Academy Rd \n \n,210,"237,000","237,000","334,604","237,000",7 Academy Rd \n \n,210,"334,000","334,000","334,000","334,000"
4,75.36-2-76,9 Academy Rd \n \n,210,"192,000","192,000","271,072","168,080",9 Academy Rd \n \n,210,"359,000","359,000","359,000","329,000"


In [11]:
#drop columns not needed
merged_data = merged_data.drop(columns=['address_2024', 'property_type_code_2024'])
merged_data.head()

,parcel_num,address_2023,property_type_code_2023,county_tax_2023,city_tax_2023,full_market_value_2023,school_tax_2023,county_tax_2024,city_tax_2024,full_market_value_2024,school_tax_2024
0,76.22-3-40,Rear 335.5 Myrtle Ave,311,"1,400","1,400","1,977","1,400","3,000","3,000","3,000","3,000"
1,75.36-2-79,3 Academy Rd \n \n,210,"205,000","205,000","289,425","205,000","378,000","378,000","378,000","378,000"
2,75.36-2-78,5 Academy Rd \n \n,210,"162,000","90,000","254,130","97,110","149,000","149,000","298,000","65,000"
3,75.36-2-77,7 Academy Rd \n \n,210,"237,000","237,000","334,604","237,000","334,000","334,000","334,000","334,000"
4,75.36-2-76,9 Academy Rd \n \n,210,"192,000","192,000","271,072","168,080","359,000","359,000","359,000","329,000"


In [12]:
#Save data to parquet file locally
merged_data.to_parquet('parsed_data/albany_assessment_2023_and_2024.parquet', index=False)